# Preprocesamiento de datos

## Ejercicios: Parte I

<div class="alert-warning">
    
Indicaciones: Los ejercicios 1, 2, 3 y 4 deben leer su data de links en Github y el código y los resultados deben ser publicados en páginas en Github.

</div>

## Ejercicio 1

<div class="alert-success">
    
1. Use two maps of points.

2. Compute the distance matrix for both maps.

3. Select one row of the distance matrix, and plot the two points with the minimal distance on top of the country of your choosing. (distancia entre punto a punto)
</div>

Mapas de puntos: Plantas de energía solar y eólica en China

In [9]:
import pandas as pd
import geopandas as gpd
import os

PowerPlants = pd.read_csv("https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/data/global_power_plant_database.csv", dtype={'other_fuel3': 'object'})

China_PowerPlants = PowerPlants[PowerPlants["country"]=="CHN"]
China_PowerPlants = China_PowerPlants.loc[:,["name","gppd_idnr","capacity_mw","latitude","longitude","primary_fuel"]]
China_PowerPlants.reset_index(drop=True, inplace=True)

#Dataframe a GeoDataframe:
China_PowerPlants_gpd = gpd.GeoDataFrame(data=China_PowerPlants.copy(),
                                        geometry=gpd.points_from_xy(x=China_PowerPlants.longitude,
                                        y=China_PowerPlants.latitude),
                                        crs=4326)

#Cambiando los CRSs por unos proyectados y que coincidan con los de China3415:
China3415_PowerPlants = China_PowerPlants_gpd.to_crs(3415)

#Mapas de puntos: Plantas de energía solar y eólica
China3415_SolarPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Solar"]
China3415_SolarPowerPlants = China3415_SolarPowerPlants.rename(columns={"name":"SolarPowerPlant_Name"})
China3415_SolarPowerPlants = China3415_SolarPowerPlants.reset_index()
China3415_SolarPowerPlants.drop(columns="index",inplace=True)


China3415_WindPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Wind"]
China3415_WindPowerPlants = China3415_WindPowerPlants.rename(columns={"name":"WindPowerPlant_Name"})
China3415_WindPowerPlants = China3415_WindPowerPlants.reset_index()
China3415_WindPowerPlants.drop(columns="index",inplace=True)

# Geopackage con los 2 mapas de puntos:
China3415_SolarPowerPlants.to_file(os.path.join("maps","China3415_PowerPlants","China3415_PowerPlants.gpkg"), layer='Solar', driver="GPKG",index="")
China3415_WindPowerPlants.to_file(os.path.join("maps","China3415_PowerPlants","China3415_PowerPlants.gpkg"), layer='Wind', driver="GPKG",index="")

## Ejercicio 2

<div class="alert-success">
    
1. Use a map of points and a map of lines.

2. Compute the distance matrix for both maps.

3. Select one line of the distance matrix, and plot the closests and the farthest point to that line. (Se puede seleccionar un río o un sistema y calcular el punto más cercano o lejano a la línea. Si se escoge un punto, se calcula la línea más cercana y lejana)
    
    
</div>

Ríos principales en China:
Fuente: [Major Rivers in the World](https://datacatalog.worldbank.org/search/dataset/0042032/Major-Rivers-of-the-World)

In [17]:
MajorRiverslink = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/World_MajorRivers/MajorRivers.shp"
MajorRivers = gpd.read_file(MajorRiverslink)
MajorRivers3415 = MajorRivers.to_crs(3415)

China3415_MajorRivers = gpd.clip(gdf = MajorRivers3415, mask = China3415)
China3415_MajorRivers = China3415_MajorRivers.reset_index()
China3415_MajorRivers = China3415_MajorRivers.drop(columns="index")
China3415_MajorRivers

China3415_MajorRivers.to_file(os.path.join("maps","China3415","ChinaMaps_3415.gpkg"), layer='MajorRivers', driver="GPKG",index="")

NameError: name 'China3415' is not defined

Plantas de energía nuclear en China:

## Ejercicio 3

<div class="alert-success">
    
1. Create a set of points and a set of polygons

2. Compute the distance matrix for both sets.

3. Select one polygon of the distance matrix, and plot the closests and the farthest point to that polygon. (cualquiera de los dos mapas. Puede ser el mapa estático o el mapa interactivo)
    
</div>   

Conjunto de polígonos:

Conjunto de puntos: Plantas de energía nuclear en China

In [13]:
China3415_NuclearPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Nuclear"]
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.rename(columns={"name":"NuclearPowerPlant_Name"})
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.reset_index()
China3415_NuclearPowerPlants.drop(columns="index",inplace=True)

China3415_NuclearPowerPlants.to_file(os.path.join("maps","China3415_PowerPlants","China3415_PowerPlants.gpkg"), layer='Nuclear', driver="GPKG",index="")

## Ejercicio 4

<div class="alert-success">
    
1. Create a set of points and a set of lines

2. Get the buffer for the lines, select different values for the distance. Jugar con varias distancias, pero solo colocar una que tenga resultados interesantes en el notebook final.

3. Keep the points that are within the buffer (as in point 2, you need to play with differn distances until you show something interesting. Todos los puntos que están dentro del buffer creado para la línea.
   

</div>   

Conjunto de líneas: